<h3>Importing the environment variables</h3>

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

<h3>Connecting and loading the Database</h3>

In [3]:
from langchain_community.utilities.sql_database import SQLDatabase

In [4]:
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_name = os.getenv("db_name")

In [5]:
mysql_uri = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"

db = SQLDatabase.from_uri(mysql_uri , sample_rows_in_table_info = 4)

In [7]:
print(db.get_usable_table_names())
# print(db.table_info)

['attendance', 'login', 'students', 'subjects', 'teacher_subject', 'teachers']


<h3>Creating the model</h3>

In [8]:
from langchain.chains import create_sql_query_chain
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq

In [9]:
llm = ChatOllama(model="llama3.1" , temperature= 0)

In [10]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
)

<h4>Creating the first phase of operation - Generating the query and executing it</h4>

<p>Creating the query generator</p>

In [ ]:
#The very first step in our interaction
generate_query = create_sql_query_chain(llm , db)       #It also needs a final prompt which we will give later

In [12]:
suffix = " Only return the SQL query, Nothing else."

In [13]:
query = generate_query.invoke({"question" : "How many students are there?" + suffix})
print(query)

SELECT COUNT(*) AS total_students FROM students


<p>Creating the SQL Executor</p>

In [14]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [15]:
execute_query = QuerySQLDataBaseTool(db = db)
execute_query.invoke(query)

'[(284,)]'

<p>Chaining the Query generator and Executor</p>

In [16]:
#Creating the chain of above 2 steps
chain = generate_query | execute_query

In [17]:
chain.invoke({"question" : "How many students are there?" + suffix})

'[(284,)]'

<h4>Creating the second phase of operation - Feeding the obtained result to generate Natural Language Response</h4>

<p>Creating an answer prompt to give to second phase llm for natural responses</p>

In [18]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [19]:
answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question in natural english language.If the result is blank, then answer in negative.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

<p>Chaining the answer prompt to llm and the extracting the response as plain old String</p>

In [20]:
rephrase_answer = answer_prompt | llm | StrOutputParser()

<p>Appending this chain to our operational chain</p>

In [21]:
chain = (
     RunnablePassthrough.assign(query=generate_query).assign(
         result=itemgetter("query") | execute_query
     )
     | rephrase_answer
 )

In [22]:
chain.invoke({"question": "How many students?" + suffix})

'There are 284 students.'

<h3>Creating Few-Shot Prompts</h3>

In [26]:
examples = [                                #These examples behave like training set to llm making it to believe we had a conversation like this
    {
        "input" : "List all students of the class" , 
        "query" : "SELECT * FROM students;"
    } ,
    {
        "input" : "Give me the usn of all students" , 
        "query" : "SELECT student_id FROM students;"
    } ,
    {
        "input" : "Give me details of student id 106" , 
        "query" : "SELECT * FROM students WHERE student_id = '1DS21CS106'"
    } ,
    {
        "input" : "Who is 087?" , 
        "query" : "SELECT * FROM students WHERE student_id = '1DS21CS087'"
    } ,
    {
        "input" : "Give me details of student id 1ds21cs087" , 
        "query" : "SELECT * FROM students WHERE student_id = '1DS21CS087'"
    } ,
    {
        "input" : "Give me details of usn 234" , 
        "query" : "SELECT * FROM students WHERE student_id = '1DS21CS234'"
    } ,
    {
        "input" : "Give me details of usn 1ds21cs234" , 
        "query" : "SELECT * FROM students WHERE student_id = '1DS21CS234'"
    } , 
    {
        "input" : "Who teaches ar vr?" ,
        "query" : "SELECT teacher_name FROM teachers INNER JOIN teacher_subject ON teachers.teacher_id = teacher_subject.teacher_id INNER JOIN subjects ON teacher_subject.subject_id = subjects.subject_id WHERE subjects.subject_name = 'AR VR';"
    } , 
    {
        "input" : "Who teaches business intelligence?" ,
        "query" : "SELECT teacher_name FROM teachers INNER JOIN teacher_subject ON teachers.teacher_id = teacher_subject.teacher_id INNER JOIN subjects ON teacher_subject.subject_id = subjects.subject_id WHERE subjects.subject_name = 'Business Intelligence';"
    } ,
    {
        "input" : "What Premsukh teaches?" ,
        "query" : "select su.subject_name from teachers t inner join teacher_subject ts on t.teacher_id = ts.teacher_id inner join subjects su on su.subject_code = ts.subject_code where t.teacher_id in (select teacher_id from teachers where teacher_name like '%premsukh%'); "
    } ,
    {
        "input" : "what Harsh teaches?" , 
        "query" : "select su.subject_name from teachers t inner join teacher_subject ts on t.teacher_id = ts.teacher_id inner join subjects su on su.subject_code = ts.subject_code where t.teacher_id in (select teacher_id from teachers where teacher_name like '%harsh%'); "
    } ,
    {
        "input" : "How many subjects Tanmayi teaches?" ,
        "query" : "SELECT COUNT(*) FROM teacher_subject ts WHERE teacher_id IN (SELECT teacher_id FROM teachers t WHERE t.teacher_name LIKE '%tanmayi%');"
    } ,
]

<p>Creating Prompt Templates</p>

In [27]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder , FewShotChatMessagePromptTemplate

In [29]:
example_prompt = ChatPromptTemplate.from_messages(              #The above examples will be presented to llm in this format
    [
        # ("human" , "{input}\nSQLQuery:"),
        ("human" , "{input}. " + suffix),
        ("ai" , "{query}"), 
    ]
)

#Mind you, this is just a format. It does not contain the above examples yet.

In [30]:
#This template will connect the above examples and the format speicified above.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt ,
    examples= examples ,
    input_variables=["input"],                  
)

In [31]:
print(few_shot_prompt.format(input="How many students are there?"))

Human: List all students of the class.  Only return the SQL query, Nothing else.
AI: SELECT * FROM students;
Human: Give me the usn of all students.  Only return the SQL query, Nothing else.
AI: SELECT student_id FROM students;
Human: Give me details of student id 106.  Only return the SQL query, Nothing else.
AI: SELECT * FROM students WHERE student_id = '1DS21CS106'
Human: Who is 087?.  Only return the SQL query, Nothing else.
AI: SELECT * FROM students WHERE student_id = '1DS21CS087'
Human: Give me details of student id 1ds21cs087.  Only return the SQL query, Nothing else.
AI: SELECT * FROM students WHERE student_id = '1DS21CS087'
Human: Give me details of usn 234.  Only return the SQL query, Nothing else.
AI: SELECT * FROM students WHERE student_id = '1DS21CS234'
Human: Give me details of usn 1ds21cs234.  Only return the SQL query, Nothing else.
AI: SELECT * FROM students WHERE student_id = '1DS21CS234'
Human: Who teaches ar vr?.  Only return the SQL query, Nothing else.
AI: SELECT

<p>If you notice , there can be a lot of examples which may be irrelevant to question asked, so we plan to pass only relevant examples <b><i>dynamically</i></b> to the llm</p>

<p>Creating dynamic few shot selections</p>

In [34]:
#Dynamic few-shot selections
from langchain_chroma import Chroma                                                     #To store the vector Database
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_ollama import OllamaEmbeddings                                           #To embed the exampels

In [35]:
vectorstore = Chroma()
vectorstore.delete_collection()                                                #Clearing the contents of database beforehand

In [ ]:
#We will create a method which will dynamically select the most relevent and similar in semantics to the question asked
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OllamaEmbeddings(model="llama3.1",),
    vectorstore,
    k=2,
    input_keys=["input"],
)

In [41]:
example_selector_input = "Who teaches software atchitecture?"

In [42]:
example_selector.select_examples({"input": example_selector_input})

[{'input': 'Who teaches business intelligence?',
  'query': "SELECT teacher_name FROM teachers INNER JOIN teacher_subject ON teachers.teacher_id = teacher_subject.teacher_id INNER JOIN subjects ON teacher_subject.subject_id = subjects.subject_id WHERE subjects.subject_name = 'Business Intelligence';"},
 {'input': 'Who teaches ar vr?',
  'query': "SELECT teacher_name FROM teachers INNER JOIN teacher_subject ON teachers.teacher_id = teacher_subject.teacher_id INNER JOIN subjects ON teacher_subject.subject_id = subjects.subject_id WHERE subjects.subject_name = 'AR VR';"}]

<p>In case of curiosity , how embeddings work, Run this</p>

In [44]:
# embed = OllamaEmbeddings(
#     model="llama3.1"
# )

# input_text = "Who teaches Generative AI?"
# vector = embed.embed_query(input_text)
# print(vector[:3])

# input_text = "Who teaches business intelligence?"
# vector = embed.embed_query(input_text)
# print(vector[:3])

<p>We will pass this method to our earlier defined few_shot_prompt so that it can dynamically pass the relevant examples to the llm</p>

In [46]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,                          #The format     
    example_selector=example_selector,                      #The dynamic few shot selector
    input_variables=["input", "top_k"],                     
)

In [47]:
few_shot_prompt_input = "Who teaches management entrepreneurship?"

In [48]:
print(few_shot_prompt.format(input=few_shot_prompt_input))

Human: Who teaches business intelligence?.  Only return the SQL query, Nothing else.
AI: SELECT teacher_name FROM teachers INNER JOIN teacher_subject ON teachers.teacher_id = teacher_subject.teacher_id INNER JOIN subjects ON teacher_subject.subject_id = subjects.subject_id WHERE subjects.subject_name = 'Business Intelligence';
Human: Who teaches ar vr?.  Only return the SQL query, Nothing else.
AI: SELECT teacher_name FROM teachers INNER JOIN teacher_subject ON teachers.teacher_id = teacher_subject.teacher_id INNER JOIN subjects ON teacher_subject.subject_id = subjects.subject_id WHERE subjects.subject_name = 'AR VR';


<p>Now we will merge all our templates and make it a final one to pass to llm</p>

In [54]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [55]:
final_prompt_input = "Who is 098?"

In [56]:
print(final_prompt.format(input=final_prompt_input,table_info=db.table_info))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: 
CREATE TABLE attendance (
	student_id VARCHAR(15) NOT NULL, 
	subject_id VARCHAR(15) NOT NULL, 
	date DATE NOT NULL, 
	status ENUM('absent','present') DEFAULT 'absent', 
	PRIMARY KEY (student_id, subject_id, date), 
	CONSTRAINT attendance_ibfk_1 FOREIGN KEY(student_id) REFERENCES students (student_id), 
	CONSTRAINT attendance_ibfk_2 FOREIGN KEY(subject_id) REFERENCES subjects (subject_id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
4 rows from attendance table:
student_id	subject_id	date	status
1DS21CS001	21CS71	2024-11-16	absent
1DS21CS001	21CS72	2024-11-16	present
1DS21CS001	21CS73	2024-11-16	absent
1DS21CS001	21CS74	2024-11-16	present
*/


CREATE TABLE login (
	id VARCHAR(20) NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(100) NOT NULL, 
	password VARCHAR(255) NOT NULL, 
	type VA

<p>As told earlier , the final prompt will be give to the query generator. Now is the time. </p>

In [57]:
generate_query = create_sql_query_chain(llm, db,final_prompt)

<p>Update the chain as well</p>

In [59]:
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)

In [60]:
chain.invoke({"question" : "How many students are there?"})

'There are 284 students.'

<h3>Adding memory to the chat</h3>

In [61]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

<p>Tweaking the final prompt to add the past conversation</p>

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for reference and may be considered while answering follow up questions"),
         few_shot_prompt,
         MessagesPlaceholder(variable_name="messages"),             #adding the past conversation if happened.
         ("human", "{input}"),
     ]
 )

In [64]:
print(final_prompt.format(input=final_prompt_input,table_info=db.table_info,messages=[]))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: 
CREATE TABLE attendance (
	student_id VARCHAR(15) NOT NULL, 
	subject_id VARCHAR(15) NOT NULL, 
	date DATE NOT NULL, 
	status ENUM('absent','present') DEFAULT 'absent', 
	PRIMARY KEY (student_id, subject_id, date), 
	CONSTRAINT attendance_ibfk_1 FOREIGN KEY(student_id) REFERENCES students (student_id), 
	CONSTRAINT attendance_ibfk_2 FOREIGN KEY(subject_id) REFERENCES subjects (subject_id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
4 rows from attendance table:
student_id	subject_id	date	status
1DS21CS001	21CS71	2024-11-16	absent
1DS21CS001	21CS72	2024-11-16	present
1DS21CS001	21CS73	2024-11-16	absent
1DS21CS001	21CS74	2024-11-16	present
*/


CREATE TABLE login (
	id VARCHAR(20) NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(100) NOT NULL, 
	password VARCHAR(255) NOT NULL, 
	type VA

<p>As we updated the final prompt , so we have to update the query generator as well as the chain again</p>

In [65]:
generate_query = create_sql_query_chain(llm, db,final_prompt)

In [66]:
chain = (
 RunnablePassthrough.assign(query=generate_query).assign(
     result=itemgetter("query") | execute_query
 )
 | rephrase_answer
 )

In [67]:
history.clear()                 #To clear the past conversation

In [68]:
print(history.messages)

[]


In [70]:
question = "How many students are there with name starting with p?"

In [71]:
response = chain.invoke({"question": question,"messages":history.messages})
print(response)

There are 27 students with names starting with the letter 'p'.


<p>Adding this conversation</p>

In [72]:
history.add_user_message(question)
history.add_ai_message(response)

In [74]:
history.messages

[HumanMessage(content='How many students are there with name starting with p?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="There are 27 students with names starting with the letter 'p'.", additional_kwargs={}, response_metadata={})]

In [75]:
response = chain.invoke({"question": "Can you give me name of first 5?","messages":history.messages})
print(response)

Sure, the first five names starting with 'P' are PILLA SHANMUKHA DHANUSH, POOJA MARIA, POORNESH KAMATH S, POTANA SAI ROHITH, and PRAGYA SHARMA.
